# Introduction

As part of this 'sandbox' environment, an [EPICS](http://www.aps.anl.gov/epics/) IOC is running here and available for interaction. This IOC provides PVs (process variables) to interact with 6 independent motors, an [areadetector](http://cars9.uchicago.edu/software/epics/areaDetector.html) camera producing 1024x1024 images, and 6 point-detectors. These point-sensors yield scalar values whose amplitude depends the position of an associated simulated motor. The features produced by the point sensors are found in the motor ranges -5 to 5 units.

The pv names and descriptions are here:

| PV Name | Description |
| :-----: | :-------:   |
|sim:mtr1 | Motor Record 1 |
|sim:mtr2 | Motor Record 2 |
|sim:mtr3 | Motor Record 3 |
|sim:mtr4 | Motor Record 4 |
|sim:mtr5 | Motor Record 5 |
|sim:mtr6 | Motor Record 6 |
| ------- | -------        |
|sim:dev1:curr | scaler sensor 1; linked to 'sim:mtr1'; peak value at 1.5 |
|sim:dev2:curr | scaler sensor 2; linked to 'sim:mtr2'; peak value at 0.5 |
|sim:dev3:curr | scaler sensor 3; linked to 'sim:mtr3'; peak value at -0.5 |
|sim:dev4:curr | scaler sensor 4; linked to 'sim:mtr4'; peak value at -1.5 |
|sim:dev5:curr | scaler sensor 5; linked to 'sim:mtr5'; peak value at -2.5 |
|sim:dev6:curr | scaler sensor 6; linked to 'sim:mtr6'; peak value at -1.0 |
| ------- | -------        |
|sim:det: | areaDetector pv prefix; available plugins include 'image1:', 'cam1:', 'Stats1-5', etc|

With this set of simulated hardware to interact with, we can exercise Ophyd, collect data with Bluesky, and retrieve data via the DataBroker for analysis and inspection.

### Basic EPICS tools

The familiar EPICS commandline tools _caget_ and _caput_ are available. These provide a simple means of communicating with the simulated hardware.

In [ ]:
from epics import caget, caput

In [ ]:
# fetch motor 1's readback position
caget('sim:mtr1.RBV')

In [ ]:
# move motor 1 to a new position
caput('sim:mtr1', 3.14)

from time import sleep
sleep(3.0) # give motor 1 time to move

In [ ]:
# check motor 1's new position
caget('sim:mtr1.RBV')

In [ ]:
# what's the value of sensor 1 at the current position of motor 1?
caget('sim:dev1:curr')

### Initialize Ophyd

Prior to making any other calls into the Ophyd python package, it should be initialized. This will provide a safe context for EPICS communication.

In [ ]:
from ophyd.commands import setup_ophyd
setup_ophyd()

### Devices

Ophyd permits working with _Devices_, rather than with individual EPICS process variables.

In [ ]:
from ophyd import EpicsMotor, EpicsSignalRO

m1 = EpicsMotor('sim:mtr1', name='m1')
m2 = EpicsMotor('sim:mtr2', name='m2')

# connect to sensor 1 as a read-only Signal
sig1 = EpicsSignalRO('sim:dev1:curr', name='sig1')

In [ ]:
# are both EpicsMotors fully connected?
m1.connected and m2.connected

In [ ]:
m1.position

Now we can more naturally _move_ a motor. In this case, we will wait for the move to complete rather than proceeding to the next instructions immediately.

In [ ]:
m1.move(0, wait=True)

In [ ]:
m1.position

In [ ]:
# we can even do a primitive scan using motor 1 and reading sensor 1 at each position
import numpy as np

positions = []
data = []

for pos in np.linspace(-5, 5, 25):
    status = m1.move(pos)
    while not status.done: pass
    positions.append(m1.position)
    data.append(sig1.value)

%matplotlib inline
from matplotlib.pyplot import plot, show

plot(positions, data)

We will see in other tutorials how Bluesky makes the process of data collection _much_ more powerful than what was achieved here.